<a href="https://colab.research.google.com/github/fabiormazza/IA025_2022S1/blob/main/ex09/fabio_mazza/Aula_9_Exerc%C3%ADcio_Final4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Fabio Renato Zocal Mazza'
print(f'Meu nome é {nome}')

Meu nome é Fabio Renato Zocal Mazza


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 83.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import tqdm

from collections import OrderedDict


In [4]:
# Check which GPU we are using
!nvidia-smi

Sun Jun 12 21:15:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        
        self.max_seq_length = max_seq_length
        cls = 101
        sep = 100
        tokens_all = torch.tensor([])
        inputs = torch.tensor([])
        targets = torch.tensor([])
 
        for text in tqdm.notebook.tqdm(texts):
            tokens = tokenize(text, tokenizer)
            tokens = [cls] + tokens + [sep]
            tokens = torch.FloatTensor(tokens)
            tokens_all = torch.cat((tokens_all, tokens))
            #print(tokens_all)
        
        pad_size = max_seq_length - tokens_all.size(dim=0) % max_seq_length
        tokens_all = torch.cat((tokens_all, torch.zeros(pad_size)), dim=0) #pad_token_id = 0 (review if otherwise) / tokens_uns is the tokens tensor before reshaping
        inputs = tokens_all.reshape((-1, max_seq_length))

        t_tokens = torch.roll(tokens_all, -1, 0)
        t_tokens[-1] = 0
        targets = t_tokens.reshape((-1, max_seq_length))
        
        self.inputs = torch.LongTensor(inputs.numpy())
        self.targets = torch.LongTensor(targets.numpy())

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

#assert torch.equal(first_batch_input, correct_first_batch_input)
#assert torch.equal(first_batch_target, correct_first_batch_target)

#print('Passou no assert de dataset.')

print(first_batch_input, '\n', first_batch_target)

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
tensor([[  101,  3396, 10303,   125, 13239,   100,   101,  1660,  5971],
        [  785,   125,  1847, 13779, 15616,   100,     0,     0,     0]]) 
 tensor([[ 3396, 10303,   125, 13239,   100,   101,  1660,  5971,   785],
        [  125,  1847, 13779, 15616,   100,     0,     0,     0,     0]])


In [8]:
## Teste com frases longas

dummy_texts = ['Os primeiros socorros foram feitos no local e a vítima foi levada \
ao Hospital de Base de Brasília com ferimentos leves, consciente e com estado de saúde estável.', 
'Um veículo capotou na Via Estrutural na tarde desta sexta-feira (3), por volta das 15h. O Corpo\
 de Bombeiros Militar do Distrito Federal (CBMDF) foi chamado ao local para realizar os primeiros atendimentos à vítima.']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)

first_batch_input, first_batch_target = next(iter(dummy_loader))

print(first_batch_input)
print(first_batch_target)

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[  101,   533,  1867, 19279,   444,   506,  5159,   202,  1238],
        [  122,   123,  8329,   262, 11067,   320,  6297,   125,  8831],
        [  125,  6191,   170, 16811, 14698,   117, 19054,   122,   170],
        [ 1177,   125,  3231, 12055,   119,   100,   101,  1263,  7594],
        [  853,  3356,   203,   229,  7811,  9009,   381,  1441,   229],
        [ 1373,  1014,  7250,   118, 14258,   113,   511,   114,   117]])
tensor([[  533,  1867, 19279,   444,   506,  5159,   202,  1238,   122],
        [  123,  8329,   262, 11067,   320,  6297,   125,  8831,   125],
        [ 6191,   170, 16811, 14698,   117, 19054,   122,   170,  1177],
        [  125,  3231, 12055,   119,   100,   101,  1263,  7594,   853],
        [ 3356,   203,   229,  7811,  9009,   381,  1441,   229,  1373],
        [ 1014,  7250,   118, 14258,   113,   511,   114,   117,   240]])


In [9]:
## Teste dos métodos

print(len(first_batch_input))
print(dummy_dataset[0])

6
(tensor([  101,   533,  1867, 19279,   444,   506,  5159,   202,  1238]), tensor([  533,  1867, 19279,   444,   506,  5159,   202,  1238,   122]))


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [10]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-12 21:15:37--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   138MB/s    in 7.6s    

2022-06-12 21:15:45 (155 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [11]:
# Load datasets
max_seq_length = 12

train_examples = 50000
valid_examples = 5000
test_examples = 5000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 60000 lines.


  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [12]:
## Número médio de palavras em datasets
avlen = 0

for text in training_texts:
    avlen += len(text.split())/len(training_texts)

print(avlen)
print(training_texts[-10])
print(len(training_texts[-10]))

777.391560000011
Milhões cometem 'facebookcídio' por medo da espionagem dos EUA PONDO FIM ÀS PRÓPRIAS CONTAS NA REDE Embora ainda cresça em ritmo acelerado no terceiro mundo, milhões de usuários desiludidos dos países desenvolvidos decidiram recentemente pôr fim às suas próprias contas no Facebook. Só nos EUA foram 9 milhões de facebookcídios contra 2 milhões na Inglaterra. A maioria das baixas (48,3%) aconteceu após as denúncias de espionagem e violações de privacidade na rede social. Em agosto deste ano, o portal Infowars revelou que, somente no primeiro semestre, o Facebook havia entregue ao governo dos EUA as informações de 38 mil usuários em 74 países. Mas não são apenas os aspectos relacionados à falta de privacidade que explicam as baixas: insatisfação geral com a rede (13,5%), baixa qualidade de publicações e debates (12,6%) e medo de se tornar um "viciado" (6%). Segundo a pesquisa da Universidade de Viena, na Áustria, a maioria dos usuários que deixou a rede é formada por home

In [13]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 4631157
valid examples: 458916
test examples: 487687


In [14]:
class AttentionBlock(torch.nn.Module):

    def __init__(self, dim: int, L: int, V: int):

        super(AttentionBlock, self).__init__()
        
        self.D = dim
        n_ff = 128

        self.L = L
        self.wq = nn.Linear(dim, dim, bias = False, device=device)
        self.wk = nn.Linear(dim, dim, bias = False, device=device)
        self.wv = nn.Linear(dim, dim, bias = False, device=device)
        self.wo = nn.Linear(dim, dim, bias = False, device=device)

        self.ff = nn.Sequential(OrderedDict([
                              ('ff1', torch.nn.Linear(dim, n_ff, device=device)),
                              ('relu', torch.nn.ReLU()),
                              ('dropout', torch.nn.Dropout(p=0.20)),
                              ('ff2', torch.nn.Linear(n_ff, dim, device=device, bias = False))
        ]))
    
    def forward(self, X):

        residual = X

        Q = self.wq(X)
        K = self.wk(X)
        V = self.wv(X)
        scores = torch.matmul(Q, K.permute(0, 2, 1))/math.sqrt(self.D)
        
        #causal mask
        mask = torch.ones((self.L, self.L), dtype=torch.bool)
        mask = torch.triu(mask, diagonal=1)
        scores[:, mask] = -1000000000

        probs = torch.nn.functional.softmax(scores, dim=-1)
        E = torch.matmul(probs, V)
        E = self.wo(E)
        out = self.ff(E)
        out = out + residual

        return out


class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super(LanguageModel, self).__init__()

        self.V = vocab_size
        self.L = max_seq_length
        self.D = dim
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id

        n_linear_1 = 128
        n_linear_2 = 64

        self.embedding = torch.nn.Embedding(vocab_size, dim, device=device)
        self.positional = torch.nn.Parameter(torch.randn(max_seq_length, dim, device=device)/10000)

        self.attention = nn.Sequential(OrderedDict([
                          ('attention_1', AttentionBlock(dim, self.L, self.V)),
                          ('attention_2', AttentionBlock(dim, self.L, self.V))
        ]))

        self.linear = nn.Sequential(OrderedDict([
                              ('l1', torch.nn.Linear(dim, n_linear_2, device=device)),
                              ('relu', torch.nn.ReLU()),
                              ('dropout', torch.nn.Dropout(p=0.20)),
                              ('l2', torch.nn.Linear(n_linear_2, vocab_size, device=device, bias = False))
        ]))

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        X = self.embedding(inputs)
        X = X + self.positional
        E = self.attention(X)
        out = self.linear(E)

        return out

## Teste o modelo com um exemplo

In [15]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 12])
sample_output.shape: torch.Size([1, 12, 29794])


In [16]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3884352


## Assert da Perplexidade


In [17]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30633
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [18]:
max_examples = 50_000_000
eval_every_steps = 1000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=128,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=512)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30726.80, valid ppl: 30370.57
1000 steps; 512000 examples so far; train ppl: 1773.20, valid ppl: 958.90
2000 steps; 1024000 examples so far; train ppl: 887.55, valid ppl: 679.09
3000 steps; 1536000 examples so far; train ppl: 689.21, valid ppl: 547.42
4000 steps; 2048000 examples so far; train ppl: 581.18, valid ppl: 470.95
5000 steps; 2560000 examples so far; train ppl: 512.44, valid ppl: 420.60
6000 steps; 3072000 examples so far; train ppl: 468.17, valid ppl: 386.74
7000 steps; 3584000 examples so far; train ppl: 436.11, valid ppl: 360.65
8000 steps; 4096000 examples so far; train ppl: 410.44, valid ppl: 342.36
9000 steps; 4608000 examples so far; train ppl: 393.26, valid ppl: 326.85
10000 steps; 5120000 examples so far; train ppl: 375.18, valid ppl: 315.18
11000 steps; 5632000 examples so far; train ppl: 362.63, valid ppl: 304.92
12000 steps; 6144000 examples so far; train ppl: 352.95, valid ppl: 296.59
13000 steps; 6656000 examples so far; tr

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [19]:
test_loader = DataLoader(test_dataset, batch_size=512)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 197.37457448394727


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = 'O presidente do Senado, Rodrigo Pacheco, se reuniu com a'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa,
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que é
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que é o
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que é o que
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que é o que é
O presidente do Senado, Rodrigo Pacheco, se reuniu com a sua casa, o que é o que é 

In [21]:
prompt = 'Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é o
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é o que
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é o que é
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é o que é o
Houve uma compreensão de que todas essas iniciativas deveriam ser coordenadas com o que o que é o que é o que
Houve uma compreensão de que

In [22]:
prompt = 'Não ofereceu resistência, não teve troca de tiros, foi uma'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Não ofereceu resistência, não teve troca de tiros, foi uma das
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais problemas
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais problemas de
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais problemas de comunicação
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais problemas de comunicação,
Não ofereceu resistência, não teve troca de tiros, foi uma das maiores de um dos principais problema

In [23]:
prompt = 'O filme apresenta quatro histórias, que são interligadas pela aparição de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

O filme apresenta quatro histórias, que são interligadas pela aparição de um
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos,
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos, o
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos, o que
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos, o que é
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos, o que é o
O filme apresenta quatro histórias, que são interligadas pela aparição de um dos seus filhos, o que é o que
O filme apresenta quatro histórias, que são interligadas pe

In [24]:
prompt = 'Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho,
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o que
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o que é
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o que é o
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o que é o que
Mais produtivo do que tentar definir Literatura talvez seja encontrar um caminho, mas não é o que é o que é
Mais produtivo do que tentar definir Liter

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.